In [1]:
### Dont Run All Cells , just untill trip data is saved (to_csv)
import time

start = time.time()

import pandas as pd 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os

In [2]:
base = r"/home/parth/Machine Learning/Datasets/North Corp/Ajanta/DMK"
base_ = r"/home/parth/Machine Learning/Datasets/North Corp/Ajanta"
file_list = os.listdir(base)
file = file_list[0]
file_path = os.path.join(base , file)
data = pd.read_csv(file_path,delimiter=';')

In [3]:
tripData = pd.read_csv(os.path.join(base_ , "trip" , "Trip_MC2M1PRC0GJ009702.csv"))

In [48]:
def completeDataProcessor(data , tripData):
    def getRange(d1 , d2):
        obj1 = datetime.strptime(d1 , "%Y-%m-%d %H:%M:%S")
        t1 = (obj1 - datetime(1970 , 1 , 1)).total_seconds()
        obj2 = datetime.strptime(d2 , "%d/%m/%Y %H:%M")
        t2 = (obj2 - datetime(1970 , 1 , 1)).total_seconds()
        return range(int(t1) , int(np.ceil(t2)) + 1)
    df = data.sort_values("updateddate" , ascending=True)
    df["timeObject"] = df.updateddate.apply(lambda x: datetime.strptime(x , "%Y-%m-%d %H:%M:%S"))
    df["timeSeconds"] = df.timeObject.diff().apply(lambda x : x.total_seconds())
    df.timeSeconds.at[0] = 0
    #df["avg_speed"] = df.total_distance.diff() / (df.timeSeconds / 3600.0)
    df["over_speeding"] = np.where( (df["speed"] > 60)  , 1 , 0)
    rows = []
    dfC = df.copy()
    dfC[df.over_speeding == 0] = None
    #[dfC.over_speeding.cumsum()]
    breaks = dfC.isnull().all(axis=1)
    groups = [group.dropna(how='all') for _, group in dfC.groupby(breaks.cumsum())]
    mask = [not group.empty for group in groups]
    new_groups = [group for group , m in zip(groups , mask) if m]

    useful_cols = ["latitude" , "longitude" , "updateddate"  , "timeObject"]
    for group in new_groups:
        speed = group.speed.mean()
        time = group.timeSeconds.sum()
        row = list(group[useful_cols].iloc[0].values) + [time , speed]
        rows.append(row)
    complete_dat = pd.DataFrame(data=rows , columns=useful_cols + ["Over Speeding Time" , "Speed"])
    del dfC
    tripData["OverSpeedingTime"] = None 

    for i in range(len(tripData)):
        tripData.at[i , "OverSpeedingTime"] = complete_dat[complete_dat["timeObject"].apply(
            lambda x: (x - datetime(1970 , 1 , 1)) .total_seconds() ).isin(
            getRange(tripData["Start_Date"].iloc[i] , tripData["End_Date"].iloc[i])
        )]["Over Speeding Time"].sum()

    tripData["OverSpeedingTime"] = tripData.OverSpeedingTime / 3600
    df["day"] = df["updateddate"].apply(lambda x: x.split(' ')[0])
    df["Day_Hours"] = df["updateddate"].apply(lambda x: int(x.split(' ')[1].split(':')[0]))
    from itertools import chain
    night_range = chain(range(6) , range(22 , 24))
    nt = []
    for n in night_range:
        nt.append(n)
    night_range = nt
    del nt
    day_range = range(6 , 16)
    evening_range = range(16 , 22)
    df["time_driveDiff"] = df.time_drive.diff()
    df.time_driveDiff.iloc[0] = 0

    complete_data = []
    first = True
    times = [ (a,b) for a , b in zip(tripData["Start_Date"] , tripData["End_Date"])]
    searchs = [getRange(*time) for time in times]
    tempSeconds = df.timeObject.apply(
                    lambda x:  (x - datetime(1970 , 1 , 1)).total_seconds() 
                )

    for search in searchs:
        tempSearch = df[tempSeconds.isin(search)]
        x = tempSearch[tempSearch.Day_Hours.isin(night_range)]

        night_drive = 0
        for day in x.day.unique():
            tempx = x[x.day == day]["time_driveDiff"]
            try :
                night_drive += tempx.sum()#iloc[-1] - tempx.iloc[0]
            except IndexError :
                pass


        x = tempSearch[tempSearch.Day_Hours.isin(day_range)]

        day_drive = 0
        for day in x.day.unique():
            tempx = x[x.day == day]["time_driveDiff"]
            try :
                day_drive += tempx.sum()#iloc[-1] - tempx.iloc[0]
            except IndexError :
                pass

        x = tempSearch[tempSearch.Day_Hours.isin(evening_range)]

        eve_drive = 0
        for day in x.day.unique():
            tempx = x[x.day == day]["time_driveDiff"]
            try :
                eve_drive += tempx.sum()#iloc[-1] - tempx.iloc[0]
            except IndexError :
                pass

        mini_dat = pd.DataFrame([[night_drive , day_drive , eve_drive]] , 
                                columns=["Night Drive" , "Day Drive" , "Evening Drive"])
        complete_data.append(mini_dat)

    complete_data = pd.concat(complete_data)
    df["time_idleDiff"] = df.time_idle.diff()
    df["time_idleDiff"].iloc[0] = 0
    df["isidle"] = (df["time_idleDiff"] != 0 ).astype(np.float32)
    ## Search in original data by Trip Data
    rows = []
    for search in searchs:
        # Inintialize all variables Per trip
        zerotwo = 0
        twofive = 0
        fivefift = 0
        fift = 0
        zerotwocounter = 0
        twofivecounter = 0
        fivefiftcounter = 0
        fiftcounter = 0

        # Search and Group Idle Times
        tempSearch = df[tempSeconds.isin(search)]
        tempSplit = tempSearch.copy()
        tempSplit[tempSplit.isidle == 0] = None

        breaks = tempSplit.isnull().all(axis=1)
        groups = [group.dropna(how='all') for _, group in tempSplit.groupby(breaks.cumsum())]

        for group in groups:
            #if group.shape[0] > 0:
            try :
                idleTime = group.time_idleDiff.sum() #.iloc[-1] - group.time_idle.iloc[0] 
            except IndexError :
                idleTime = 0

            idleTime = idleTime * 60
            if idleTime > 0 and idleTime <= 2 :
                zerotwocounter += 1
                zerotwo += idleTime / 60
            elif idleTime > 2 and idleTime <= 5 :
                twofivecounter+= 1
                twofive += idleTime / 60
            elif idleTime > 5 and idleTime <= 15 :
                fivefiftcounter+= 1
                fivefift += idleTime / 60
            elif idleTime > 15 :
                fiftcounter+= 1
                fift += idleTime / 60 
        row = [zerotwo , twofive , fivefift , fift , zerotwocounter ,  twofivecounter , fivefiftcounter , fiftcounter]
        rows.append(row)
    idle_portion =  pd.DataFrame(data=rows , columns=["Idle (0-2)" , "Idle (2-5)" , "Idle (5-15)" , "Idle (15 - inf)" , 
                                  "Idle (0-2) Counter" , "Idle (2-5) Counter" , "Idle (5-15) Counter" , "Idle (15 - inf) Counter"])
    idle_columns = ['Idle (0-2)', 'Idle (2-5)', 'Idle (5-15)', 'Idle (15 - inf)']
    tripData["TimeDelta"] =tripData.End_Date.apply(lambda x : datetime.strptime(x , '%d/%m/%Y %H:%M')) - tripData.Start_Date.apply(lambda x : datetime.strptime(x , '%Y-%m-%d %H:%M:%S'))
    import time
    tripData = tripData.join([complete_data.reset_index() , idle_portion])
    tripData["Stop Time"] = tripData.TimeDelta.apply(lambda x : x.total_seconds() / 3600 ) - \
    (tripData.driveTimeDelta + tripData[idle_columns].sum(axis = 1))
    end = time.time()
    tripData = tripData.drop(columns=["index"  , "TimeDelta"])
    print("Processed one file in {} Seconds".format(end-start))
    return tripData

In [47]:
tripData.to_csv("FinalTripFile.csv")